In [ ]:
#  MODEL SAVING, LOADING & REUSE

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load and preprocess data
# -----------------------------
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Normalize pixels (0–255 → 0–1)
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# Flatten 28x28 → 784
X_train = X_train.reshape(-1, 28 * 28)
X_test = X_test.reshape(-1, 28 * 28)

# Create validation set
X_val, y_val = X_train[:5000], y_train[:5000]
X_train, y_train = X_train[5000:], y_train[5000:]

#  Define model architecture
# -----------------------------
model = keras.models.Sequential([
    layers.Dense(256, activation="relu", input_shape=(784,), kernel_initializer="he_normal"),
    layers.Dense(128, activation="relu", kernel_initializer="he_normal"),
    layers.Dense(10, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

#  Train the model
# -----------------------------
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,  # small for demo
    batch_size=128,
    verbose=2
)

# Evaluate before saving
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n Original Model Accuracy: {test_acc:.4f}")

#  A. Save FULL model (.keras)
# -----------------------------
model.save("fashion_full_model.keras")
print("\n Saved full model (.keras format)")

# B. Save only weights (.h5)
# -----------------------------
model.save_weights("fashion_weights_only.weights.h5")
print(" Saved weights only (.h5)")

#  C. Export TensorFlow SavedModel (for deployment)
# -----------------------------
model.export("fashion_savedmodel_format/")
print(" Exported TensorFlow SavedModel (folder format)")

#  Delete model to simulate fresh start
# -----------------------------
del model

# Reload (A) Full .keras model
# -----------------------------
loaded_full = keras.models.load_model("fashion_full_model.keras")
print("\n Full model loaded successfully!")
print("Accuracy:", loaded_full.evaluate(X_test, y_test, verbose=0)[1])

#  Reload (B) Weights-only model
# -----------------------------
weights_only_model = keras.models.Sequential([
    layers.Dense(256, activation="relu", input_shape=(784,), kernel_initializer="he_normal"),
    layers.Dense(128, activation="relu", kernel_initializer="he_normal"),
    layers.Dense(10, activation="softmax")
])
weights_only_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Load weights
weights_only_model.load_weights("fashion_weights_only.weights.h5")
print("\n Weights-only model loaded successfully!")
print("Accuracy:", weights_only_model.evaluate(X_test, y_test, verbose=0)[1])

# Reload (C) SavedModel for inference (TFSMLayer)
# -------------------------------------------------------------
savedmodel_layer = keras.layers.TFSMLayer(
    "fashion_savedmodel_format/",
    call_endpoint="serving_default"
)

# Run inference (extract tensor from dict)
sample_preds = savedmodel_layer(X_test[:5])["output_0"]
print("\n SavedModel (TFSMLayer) loaded successfully!")
print("Predicted classes (first 5 samples):", tf.argmax(sample_preds, axis=1).numpy())




Epoch 1/5
430/430 - 3s - 7ms/step - accuracy: 0.8167 - loss: 0.5123 - val_accuracy: 0.8644 - val_loss: 0.3849
Epoch 2/5
430/430 - 2s - 4ms/step - accuracy: 0.8653 - loss: 0.3717 - val_accuracy: 0.8686 - val_loss: 0.3649
Epoch 3/5
430/430 - 2s - 5ms/step - accuracy: 0.8786 - loss: 0.3309 - val_accuracy: 0.8810 - val_loss: 0.3268
Epoch 4/5
430/430 - 2s - 5ms/step - accuracy: 0.8867 - loss: 0.3050 - val_accuracy: 0.8790 - val_loss: 0.3438
Epoch 5/5
430/430 - 2s - 5ms/step - accuracy: 0.8935 - loss: 0.2864 - val_accuracy: 0.8820 - val_loss: 0.3243

✅ Original Model Accuracy: 0.8722

💾 Saved full model (.keras format)
💾 Saved weights only (.h5)
INFO:tensorflow:Assets written to: fashion_savedmodel_format/assets


INFO:tensorflow:Assets written to: fashion_savedmodel_format/assets


Saved artifact at 'fashion_savedmodel_format/'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 784), dtype=tf.float32, name='keras_tensor_36')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  1954716774864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1954716775248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1954716775056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1954716775632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1954716773712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1954716776016: TensorSpec(shape=(), dtype=tf.resource, name=None)
💾 Exported TensorFlow SavedModel (folder format)

✅ Full model loaded successfully!
Accuracy: 0.8722000122070312

✅ Weights-only model loaded successfully!


C:\Users\chala\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\chala\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Accuracy: 0.8722000122070312

✅ SavedModel (TFSMLayer) loaded successfully!
Predicted classes (first 5 samples): [9 2 1 1 6]
